In [21]:
import pandas as pd
from datetime import datetime
from utility.helpers import DataManager
import visualization

datamanager = DataManager('provisioned', 4)

In [25]:
# Load the data
#data = helper.load_data(cluster_type='provisioned', instance_id=4)
data = pd.read_parquet("provisioned.parquet") # shortcut for developing
# Convert timestamp to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [26]:
# Visualize data
visualization.visualize_data(data)
data

,instance_id,timestamp,query_count,runtime,bytes_scanned
0,50,2024-02-29 23:00:00,1,11942.0,53912.0
1,51,2024-02-29 23:00:00,3,11015.0,8498.0
2,181,2024-02-29 23:00:00,1,29.0,26.0
3,0,2024-02-29 23:00:00,1,16546.0,0.0
4,34,2024-03-01 00:00:00,3195,3666946.0,140814.0
...,...,...,...,...,...
196034,160,2024-05-30 23:00:00,246,1978626.0,29106.0
196035,52,2024-05-30 23:00:00,914,47090168.0,101901.0
196036,183,2024-05-30 23:00:00,215,56459258.0,428348690.0
196037,11,2024-05-31 00:00:00,2,3778.0,1980.0


In [27]:
# Split into training and test data:
# Following the approach in the paper (p. 132), for a cluster with N weeks of data, 
# the first train-test split includes N-2 weeks for training and the following week 
# for testing. The second train-test split contains the first N-1 weeks for training 
# and the following week for testing, representing a scenario of re-training a model 
# each week and forecasting for the next week

train1, test1, train2, test2 = datamanager.train_test_split(data)

print(f"train1 shape: {train1.shape}")
print(f"test1 shape: {test1.shape}")
print(f"train2 shape: {train2.shape}")
print(f"test2 shape: {test2.shape}")

print(train1.iloc[-1])
print(test1.iloc[0])
print(train1.iloc[-1])
print(test1.iloc[0])

train1 shape: (173319, 5)
test1 shape: (15357, 5)
train2 shape: (188676, 5)
test2 shape: (7363, 5)
instance_id                      163
timestamp        2024-05-19 23:00:00
query_count                        2
runtime                          NaN
bytes_scanned                    NaN
Name: 190295, dtype: object
instance_id                      103
timestamp        2024-05-20 00:00:00
query_count                     4338
runtime                   40559368.0
bytes_scanned             85606319.0
Name: 43134, dtype: object
instance_id                      163
timestamp        2024-05-19 23:00:00
query_count                        2
runtime                          NaN
bytes_scanned                    NaN
Name: 190295, dtype: object
instance_id                      103
timestamp        2024-05-20 00:00:00
query_count                     4338
runtime                   40559368.0
bytes_scanned             85606319.0
Name: 43134, dtype: object


In [5]:
# Train baseline models

In [6]:
# Test baseline models and calculate error metrics

In [7]:
# visualize first prediction
visualization.visualize_prediction(train1, test1, prediction1)

# visualize first prediction
visualization.visualize_prediction(train2, test2, prediction2)

NameError: name 'prediction1' is not defined